In [1]:
import pandas as pd
import numpy as np

import os, time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

from tqdm import tqdm
from bert_utils import *
from config import *

train_df = pd.read_csv(data_path+"train.csv")
test_df = pd.read_csv(data_path+"test.csv")

# debug --
train_df = train_df.head(100)
test_df = test_df.head(100)
print("", "")
print("@@@@@@@@@@@@@@@ DEBUG with .head(100) @@@@@@@@@@@@@@@@@@")
print("", "")




# ##########################################
#
#               原文tsbの読み込み
#
# ##########################################
corpus_df_list = []
for source in ["newsplus", "news4vip", "livejupiter"]:
    corpus_df = pd.DataFrame(columns=["id", "source", "text"])
    tsv_line_list = []
    with open(input_root+f"corpus/{source}.tsv", encoding="utf-8") as f:
        for i, l in tqdm(enumerate(f)):
            tsv_line_list.append(l)
    corpus_df["text"] = tsv_line_list
    corpus_df["source"] = source
    corpus_df = corpus_df.reset_index(drop=False)
    corpus_df["id"] = corpus_df["source"] + "_" + corpus_df["index"].astype(str)
    corpus_df = corpus_df.drop(["index"], axis=1)
    corpus_df_list.append(corpus_df)

corpus_df = pd.concat(corpus_df_list)
corpus_df = corpus_df.reset_index(drop=True)


def recover_original_text(df, corpus_df):
    
    original_text_list = []
    nofinds_data_index_list = []
    duplicated_data_index_list = []
    
    for i in tqdm(range(df.shape[0]), total=df.shape[0]):
        first_sentence = df.loc[i, "first_sentence"]
        text_picked = [x for x in corpus_df["text"].values.tolist() if first_sentence in x]

        if len(text_picked) == 0: # データセットはある程度正規化されているので、データ元からヒットしないケースがある --
            nofinds_data_index_list.append(i)
            original_text_list.append("")
            continue

        elif len(text_picked) > 1: # 1つの文に対して2つ以上のレスアンカーがついていた場合, 2つ以上ヒットする --
            duplicated_data_index_list.append(i)  # 分離不能な場合があるので諦めてindexだけメモっときます --
            original_text_list.append(text_picked[0])

        else:  # 1つだけヒット
            original_text_list.append(text_picked[0])

    df["original_text"] = original_text_list
    df = df.drop(["first_sentence"], axis=1)

    return df

def recover_original_text_single(text, corpus_list):
    time.sleep(0.1)

    return_status, return_text = None, None
    text_picked = [x for x in corpus_list if text in x]
    
    if len(text_picked) == 0: # データセットはある程度正規化されているので、データ元からヒットしないケースがある --
        return_status = "No-result"
        return_text = ""

    elif len(text_picked) > 1: # 1つの文に対して2つ以上のレスアンカーがついていた場合, 2つ以上ヒットする --
        return_status = "Duplicated"
        return_text = text_picked[0]

    else:  # 1つだけヒット
        return_status = ""
        return_text = text_picked[0]

    return return_status, return_text

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

****** SEED fixed : 42 ******


 
@@@@@@@@@@@@@@@ DEBUG with .head(100) @@@@@@@@@@@@@@@@@@
 


217296it [00:00, 1213493.86it/s]
1983626it [00:01, 1390968.94it/s]
5948218it [00:03, 1508579.33it/s]


### このケースだとProcessPoolはむしろ遅くなって、ThreadPoolは多分早くなった
* そうでもないな...

In [2]:
# ########V#################################
#
#               Trainの原文検索
#
# ##########################################
train_df["first_sentence"] = train_df["text"].map(lambda x: x.split("\n")[0])
with tqdm(total=train_df.shape[0]) as progress:

    with ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = []
        for i, d in enumerate(train_df["first_sentence"]):
            future = executor.submit(recover_original_text_single, d, corpus_df["text"].values.tolist())
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)
        result = [f.result() for f in futures]

print(result)

 37%|███▋      | 37/100 [02:43<04:08,  3.95s/it]

In [ ]:
tt = pd.read_feather("input/train_df_with_original_text.feather")

In [ ]:
i = 9
print(result[i][1])
print(tt.loc[i, "original_text"])

都内の主要幹線道路の一角にいた百円で醤油の効いた海苔巻き餅を売ってくれたオジサン __BR__ あれは旨かった	磯辺焼きな __BR__ 新宿で買って食ったことあるわ	見なくなったよな __BR__ こんな話題が2ちゃんで出たのも十年は前だし __BR__ あれpayするのかな？燃料代餅代海苔代醤油代機材償却費…

都内の主要幹線道路の一角にいた百円で醤油の効いた海苔巻き餅を売ってくれたオジサン __BR__ あれは旨かった	磯辺焼きな __BR__ 新宿で買って食ったことあるわ	見なくなったよな __BR__ こんな話題が2ちゃんで出たのも十年は前だし __BR__ あれpayするのかな？燃料代餅代海苔代醤油代機材償却費…

